## Starting Model

In [133]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont


In [134]:
s3 = boto3.resource('s3')

In [135]:


def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def main():
    project_arn='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/1630820983471'
    model_arn='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130'
    min_inference_units=1 
    version_name='PropertyBot-v3-room-rekognition.2021-09-04T22.57.53'
    start_model(project_arn, model_arn, version_name, min_inference_units)

In [136]:
main()

Starting model: arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130
An error occurred (ResourceInUseException) when calling the StartProjectVersion operation: ProjectVersion arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130 is RUNNING; cannot be started
Done...


## Analyzing an Image

In [137]:
def show_custom_labels(model,bucket,photo, min_confidence):
    client=boto3.client('rekognition')

    #Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    # For object detection use case, uncomment below code to display image.
    # display_image(bucket,photo,response)

    return response['CustomLabels']

In [192]:


def label_and_save_in_bucket(prefix):
    room = ''
    bucket='propertybot-v3'

    model='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130'
    min_confidence=30

    #labeling image
    label_count=show_custom_labels(model,bucket,photo, min_confidence)
    try:
        room = label_count[0]['Name'].lower()
        print("INFO: label from model is {0}".format(room))


        # aggregating labels
        if room == "back yard" or room =="front yard":
            room = "outdoor"
        elif room == "kitchen":
            room = "kitchen"
        elif room == "garage":
            room = "garage"
        elif room == "bathroom":
            room = "bathroom"
        elif room == "living room":
            room = "other-rooms"

        print("INFO: identified a {0}".format(room))

        s3 = boto3.resource('s3')
        copy_source = {
              'Bucket': bucket,
              'Key': prefix
            }
        sink_bucket = s3.Bucket("training-images-{0}".format(room))
        sink_bucket.copy(copy_source, prefix)
        print("INFO: saved images in s3 bucket for {0}".format(room))
    except Exception as e:
        
        print(e)
 



In [191]:
# label_and_save_in_bucket(prefix="data/raw/images/Cleveland/M3142060591_109.png")

INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms


In [193]:
bucket = s3.Bucket('propertybot-v3')
count = 0
for object_summary in bucket.objects.filter(Prefix="data/raw/images/Cleveland/"):
    label_and_save_in_bucket(prefix=object_summary.key)
    count = count + 1
    if count > 100:
        break

INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-rooms
INFO: saved images in s3 bucket for other-rooms
INFO: label from model is living room
INFO: identified a other-r

KeyboardInterrupt: 